In [ ]:
import tensorflow as tf
import numpy as np
import os
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import html5lib
import pickle
import time
import progressbar

Download the news headlines for sarcasm detection dataset from Kaggle https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection/home

The dataset contains urls for the articles. Instead of just using the headlines, I wanted to pull some extra data from each of the articles. In particular, I pulled any text found within paragraph tags of the html for each article. The text data was cleaned to remove emojis and html tags.

In [ ]:
import zipfile

path = '/content/drive/MyDrive/Colab Notebooks/Sarcasm/Sarcasm_Headlines_Dataset_v2.json.zip'
z = zipfile.ZipFile(path, 'r')
z.extractall('/tmp')
z.close()


In [ ]:
import json

def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

data = list(parse_data('/tmp/Sarcasm_Headlines_Dataset_v2.json'))

In [ ]:
labels = []
headlines = []
urls = []
for record in data:
  labels.append(record['is_sarcastic'])
  headlines.append(record['headline'])
  urls.append(record['article_link'])

In [ ]:
len(urls)

28619

In [ ]:
def deEmojify(text): # thank you https://stackoverflow.com/users/6579239/abdul-razak-adam
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
def clean_html(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = ''
  for x in raw_html:
    cx = re.sub(cleanr, '', str(x))
    cx = cx.replace(u'\xa0', u' ')
    cx = cx.lstrip()
    cleantext += deEmojify(cx) + ' '
  return cleantext.lower() # the tokenizer converts to lowercase as a default

In [ ]:
content = []

with open('/content/drive/MyDrive/Colab Notebooks/Sarcasm/content.pickle', 'wb') as f:
  pickle.dump(content, f)

widgets = ['[',
         progressbar.Percentage(format='%(percentage)3d%%'),
         '] ',
           progressbar.Bar('#'),' (',
           progressbar.ETA(), ') ',
          ]

for i in progressbar.progressbar(range(len(urls)), widgets = widgets):
  req = Request(urls[i], headers={'User-Agent': 'Mozilla/5.0'})
  try:
    stream = urlopen(req)
    html = stream.read()
    stream.close()
    failures.append(False)
    html = html.decode('utf-8') 
    soup = BeautifulSoup(html, 'html5lib')
    content.append(clean_html(soup.find_all('h1') + soup.find_all('p')))
  except:
    content.append('Failed to retrieve: {}'.format(urls[i]))
    failures.append(True)
  if i % 1000 == 0:
    #reload
    with open('/content/drive/MyDrive/Colab Notebooks/Sarcasm/content.pickle', 'rb') as handle:
      C = pickle.load(handle)
    # saving
    with open('/content/drive/MyDrive/Colab Notebooks/Sarcasm/content.pickle', 'wb') as handle:
      pickle.dump(C+content, handle, protocol=pickle.HIGHEST_PROTOCOL)
    content = []

[100%] |####################################################| (Time:  4:25:16) 


In [ ]:
#reload
with open('/content/drive/MyDrive/Colab Notebooks/Sarcasm/content.pickle', 'rb') as handle:
  C = pickle.load(handle)

In [ ]:
# saving the remaining stuff not saved in loops above
with open('/content/drive/MyDrive/Colab Notebooks/Sarcasm/content.pickle', 'wb') as handle:
  pickle.dump(C+content, handle, protocol=pickle.HIGHEST_PROTOCOL)